# GPU内存层次结构 - 实践篇

本notebook通过实际代码演示GPU内存层次的使用和优化技巧。

**学习目标：**
- 对比Global Memory和Shared Memory的性能
- 掌握Shared Memory的使用方法
- 理解合并访问和Bank冲突的影响


In [ ]:
%load_ext nvcc4jupyter


## 1. 矩阵转置：Global Memory vs Shared Memory

矩阵转置是展示内存优化效果的经典示例。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// 朴素版本：直接转置，非合并写入
__global__ void transposeNaive(float* out, float* in, int width, int height) {
    int xIndex = blockIdx.x * TILE_DIM + threadIdx.x;
    int yIndex = blockIdx.y * TILE_DIM + threadIdx.y;
    
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (xIndex < width && (yIndex + j) < height) {
            // 读取合并，写入非合并（跨越stride）
            out[xIndex * height + (yIndex + j)] = in[(yIndex + j) * width + xIndex];
        }
    }
}

// 使用Shared Memory优化
__global__ void transposeShared(float* out, float* in, int width, int height) {
    __shared__ float tile[TILE_DIM][TILE_DIM + 1];  // +1避免bank冲突
    
    int xIndex = blockIdx.x * TILE_DIM + threadIdx.x;
    int yIndex = blockIdx.y * TILE_DIM + threadIdx.y;
    
    // 合并读取到Shared Memory
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (xIndex < width && (yIndex + j) < height) {
            tile[threadIdx.y + j][threadIdx.x] = in[(yIndex + j) * width + xIndex];
        }
    }
    
    __syncthreads();
    
    // 交换坐标
    xIndex = blockIdx.y * TILE_DIM + threadIdx.x;
    yIndex = blockIdx.x * TILE_DIM + threadIdx.y;
    
    // 合并写入Global Memory
    for (int j = 0; j < TILE_DIM; j += BLOCK_ROWS) {
        if (xIndex < height && (yIndex + j) < width) {
            out[(yIndex + j) * height + xIndex] = tile[threadIdx.x][threadIdx.y + j];
        }
    }
}

int main() {
    printf("==========================================\n");
    printf("    矩阵转置: Shared Memory优化对比\n");
    printf("==========================================\n\n");
    
    int width = 4096;
    int height = 4096;
    size_t size = width * height * sizeof(float);
    
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    
    // 初始化
    for (int i = 0; i < width * height; i++) {
        h_in[i] = (float)i;
    }
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);
    
    dim3 grid(width / TILE_DIM, height / TILE_DIM);
    dim3 block(TILE_DIM, BLOCK_ROWS);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float time_naive, time_shared;
    
    // 预热
    transposeNaive<<<grid, block>>>(d_out, d_in, width, height);
    transposeShared<<<grid, block>>>(d_out, d_in, width, height);
    cudaDeviceSynchronize();
    
    // 测试朴素版本
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transposeNaive<<<grid, block>>>(d_out, d_in, width, height);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_naive, start, stop);
    
    // 测试Shared Memory版本
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        transposeShared<<<grid, block>>>(d_out, d_in, width, height);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_shared, start, stop);
    
    printf("矩阵大小: %d × %d\n", width, height);
    printf("Block大小: (%d, %d)\n\n", TILE_DIM, BLOCK_ROWS);
    
    printf("朴素版本: %.3f ms\n", time_naive / 100);
    printf("Shared Memory版本: %.3f ms\n", time_shared / 100);
    printf("加速比: %.2fx\n", time_naive / time_shared);
    
    // 计算带宽
    float gb = 2.0f * width * height * sizeof(float) / 1e9;  // 读+写
    printf("\n有效带宽:\n");
    printf("  朴素版本: %.2f GB/s\n", gb / (time_naive / 100 / 1000));
    printf("  Shared Memory版本: %.2f GB/s\n", gb / (time_shared / 100 / 1000));
    
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}


## 2. Shared Memory基础用法

演示如何声明和使用共享内存。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 使用静态分配的Shared Memory
__global__ void staticSharedDemo(float* out, float* in, int n) {
    __shared__ float sdata[256];  // 静态分配：编译时确定大小
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // 加载到共享内存
    if (gid < n) {
        sdata[tid] = in[gid];
    }
    
    __syncthreads();  // 确保所有线程完成加载
    
    // 在共享内存中进行计算（示例：与相邻元素求平均）
    float result = sdata[tid];
    if (tid > 0) result = (result + sdata[tid - 1]) / 2.0f;
    
    if (gid < n) {
        out[gid] = result;
    }
}

// 使用动态分配的Shared Memory
__global__ void dynamicSharedDemo(float* out, float* in, int n) {
    extern __shared__ float sdata[];  // 动态分配：运行时确定大小
    
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (gid < n) {
        sdata[tid] = in[gid];
    }
    
    __syncthreads();
    
    float result = sdata[tid];
    if (tid > 0) result = (result + sdata[tid - 1]) / 2.0f;
    
    if (gid < n) {
        out[gid] = result;
    }
}

int main() {
    printf("==========================================\n");
    printf("        Shared Memory 基础用法\n");
    printf("==========================================\n\n");
    
    int n = 16;
    size_t size = n * sizeof(float);
    
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    
    for (int i = 0; i < n; i++) {
        h_in[i] = (float)i;
    }
    
    float *d_in, *d_out;
    cudaMalloc(&d_in, size);
    cudaMalloc(&d_out, size);
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);
    
    printf("输入数据:\n  ");
    for (int i = 0; i < n; i++) printf("%.0f ", h_in[i]);
    printf("\n\n");
    
    // 静态Shared Memory
    staticSharedDemo<<<1, 256>>>(d_out, d_in, n);
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
    printf("静态Shared Memory结果:\n  ");
    for (int i = 0; i < n; i++) printf("%.1f ", h_out[i]);
    printf("\n\n");
    
    // 动态Shared Memory - 注意第三个参数指定大小
    int sharedMemSize = 256 * sizeof(float);
    dynamicSharedDemo<<<1, 256, sharedMemSize>>>(d_out, d_in, n);
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
    printf("动态Shared Memory结果:\n  ");
    for (int i = 0; i < n; i++) printf("%.1f ", h_out[i]);
    printf("\n");
    
    printf("\n说明: 每个元素与其前一个元素取平均\n");
    
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);
    
    return 0;
}


## 3. 合并访问 vs 非合并访问

演示合并访问对Global Memory性能的影响。


In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

// 合并访问：相邻线程访问相邻内存
__global__ void coalescedAccess(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;  // 线程0访问[0], 线程1访问[1]...
    }
}

// 非合并访问：跨步访问
__global__ void stridedAccess(float* data, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int actualIdx = idx * stride;  // 跨步访问
    if (actualIdx < n) {
        data[actualIdx] = data[actualIdx] * 2.0f;
    }
}

int main() {
    printf("==========================================\n");
    printf("      合并访问 vs 非合并访问对比\n");
    printf("==========================================\n\n");
    
    int n = 32 * 1024 * 1024;  // 32M elements
    size_t size = n * sizeof(float);
    
    float *d_data;
    cudaMalloc(&d_data, size);
    
    int threadsPerBlock = 256;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    printf("数据大小: %d MB\n\n", (int)(size / (1024 * 1024)));
    
    // 预热
    coalescedAccess<<<n / threadsPerBlock, threadsPerBlock>>>(d_data, n);
    cudaDeviceSynchronize();
    
    // 测试合并访问
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        coalescedAccess<<<n / threadsPerBlock, threadsPerBlock>>>(d_data, n);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float time_coalesced;
    cudaEventElapsedTime(&time_coalesced, start, stop);
    
    // 测试不同stride的非合并访问
    printf("访问模式             时间(ms)    带宽(GB/s)\n");
    printf("------------------------------------------------\n");
    
    float gb = 2.0f * n * sizeof(float) / 1e9;  // 读+写
    printf("合并访问 (stride=1)   %7.3f     %7.2f\n", 
           time_coalesced / 100, gb / (time_coalesced / 100 / 1000));
    
    int strides[] = {2, 4, 8, 16, 32};
    for (int s = 0; s < 5; s++) {
        int stride = strides[s];
        int numElements = n / stride;
        int numBlocks = (numElements + threadsPerBlock - 1) / threadsPerBlock;
        
        // 预热
        stridedAccess<<<numBlocks, threadsPerBlock>>>(d_data, n, stride);
        cudaDeviceSynchronize();
        
        cudaEventRecord(start);
        for (int i = 0; i < 100; i++) {
            stridedAccess<<<numBlocks, threadsPerBlock>>>(d_data, n, stride);
        }
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        
        float time_strided;
        cudaEventElapsedTime(&time_strided, start, stop);
        
        float gb_strided = 2.0f * numElements * sizeof(float) / 1e9;
        printf("非合并 (stride=%2d)    %7.3f     %7.2f\n", 
               stride, time_strided / 100, gb_strided / (time_strided / 100 / 1000));
    }
    
    printf("\n结论: stride越大，有效带宽越低！\n");
    
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}


## 总结

**关键要点：**

1. **内存层次**
   - Registers > Shared Memory > L1/L2 Cache > Global Memory
   - 越靠近计算单元，速度越快，容量越小

2. **Shared Memory使用**
   - 声明: `__shared__ float data[SIZE];`
   - 必须配合 `__syncthreads()` 同步

3. **合并访问**
   - 相邻线程访问相邻地址 = 高带宽
   - 跨步访问 = 低带宽

4. **优化策略**
   - 将频繁访问的数据加载到Shared Memory
   - 保持内存访问的合并性
   - 避免Shared Memory的Bank冲突

## 练习

1. 修改矩阵转置代码，去掉`+1`的padding，观察Bank冲突对性能的影响
2. 实现一个使用Shared Memory的矩阵乘法（分块算法）
3. 尝试不同的TILE_DIM值，找到最优配置
